### Turbidity Estimation using test set sampled images

In [8]:
%matplotlib widget
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
import numpy as np
import collections
import glob
import os
import sys 
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
sys.path.insert(0, '../dip_utils')
from matrix_utils import arr_info
from vis_utils import (vis_pair, 
                       vis_triple)
%run watershed.ipynb
%run calibrate.ipynb
%run greyscale.ipynb
%run image_stacking.ipynb
%run whitep_elimination.ipynb
%run skew_correction.ipynb

In [14]:
def turbidity_estimation(dest, img, theta, h, w, ref, diff_c):
    markers = watershed(dest, img)

    # Acquire the bottom and the leftmost point of the ROI
    bottom = np.where(markers==1)[0].max()
    left = np.where((markers!=1) & (markers!=-1))[1].min()
    top = np.where(markers==2)[0].min()
    right = np.where((markers!=1) & (markers!=-1))[1].max()

    # 550 50 80 350
    vert_thres = 500
    horz_thres = 50
    length = 80
    dist = 330

    # Avoid Extracting incorrect ROI
    while markers[bottom-vert_thres, left+horz_thres] == 1:
        left += horz_thres

    # Corners: Top left, Top right, Bottom left, Bottom right
    corners = []
    tl = [left, top]
    tr = [right, top]
    bl = [left, bottom]
    br = [right, bottom]
    corners.append(tl)
    corners.append(tr)
    corners.append(bl)
    corners.append(br)

    dst_corners = get_destination_points(corners)

    # Calibration
    org = plt.imread(os.path.join(dest, img))
    org = lab_trans(clear, org)

    grey = to_single_channel(org)

    deskewed = unwarp(dest, img, np.float32(corners), dst_corners[0])
    dimensions = (dst_corners[1], dst_corners[2])
    dim = (dimensions[0] / ref[0], dimensions[1] / ref[1])
    img_dim = (int(h*dim[0]), int(w*dim[1]))
    
    img_resized = cv2.resize(grey, img_dim, interpolation=cv2.INTER_AREA)

    grey = to_single_channel(img_resized)

    # x_left represents the ROI of the reference, while x_right represents the ROI of the sample
    x_left = []
    x_right = []
    y_left = []
    y_right = []

    x = []
    x1 = []
    y = []
    y1 = []
    for i in range(dst_corners[1], dst_corners[1]-vert_thres, -1):
        x.append(i)
        y.append(np.mean(deskewed[i-1:i, horz_thres:horz_thres+length]))
    x_left.append(x)
    y_left.append(y)
    for i in range(dst_corners[1], dst_corners[1]-vert_thres, -1):
        x1.append(i)
        y1.append(np.mean(deskewed[i-1:i, horz_thres+dist:
                                         horz_thres+dist+length]))
    x_right.append(x1)
    y_right.append(y1)

    x_left = np.array(x_left)
    x_right = np.array(x_right)
    y_left = np.array(y_left)
    y_right = np.array(y_right)


    num = []
    img = img.replace('_', '.')
    a = img.split('NTU')
    a = [b.split('.jpg') for b in a]
    for item in a:
        item.remove("")
    num.append(a)

    NTU = []
    for i in num:
        for j in i:
            if len(j) != 0:
                NTU.append(float(j[0]))

    for i in range(x_left.shape[0]):
        diff = abs(np.mean(abs(y_right[i] - y_left[i])))
        diff -= diff_c
    
    estimated = round(diff*theta[0]+theta[1],2)
    return estimated, a

In [ ]:
def TE_img_list(dest, img, theta, h, w, ref, diff_c):
    if type(img) == list:
        estimated = []
        NTU = []
        for i in range(len(img)):
            e, a = turbidity_estimation(dest, img[i], theta, h, w, ref, diff_c)
            estimated.append(e)
            NTU.append(a)
        return estimated, NTU
    else:
        return turbidity_estimation(dest, img, theta, h, w, ref, diff_c)

list